In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> > > this notebook i follow the Lab code 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Input
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
import zipfile
import seaborn as sns
from os import listdir, makedirs

In [ ]:
trainData = pd.read_csv('../input/mi2023-practice/train.csv')
trainData.head()

In [ ]:
ID = trainData['Id']
images_name = trainData["Image"]
classes = pd.get_dummies(ID)
df = pd.concat([images_name,classes], axis=1)
df.head(10)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,
                             horizontal_flip=True,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             validation_split=0.2) #augmentation

In [ ]:
image_size = 299
train_dir = '../input/mi2023-practice/train/train/'
train_generator=datagen.flow_from_dataframe(
                        dataframe=trainData,
                        directory=train_dir,
                        x_col="Image",
                        y_col="Id",
                        subset="training",
                        batch_size=8,
                        seed=50,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

In [ ]:
train_dir = '../input/mi2023-practice/train/train/'
valid_generator=datagen.flow_from_dataframe(
                        dataframe=trainData,
                        directory=train_dir,
                        x_col="Image",
                        y_col="Id",
                        subset="validation",
                        batch_size=8,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

In [ ]:
testdf = pd.read_csv('../input/mi2023-practice/sample_submission.csv', dtype=str)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_dir = '../input/mi2023-practice/test/test/'
test_generator=test_datagen.flow_from_dataframe(
                            dataframe=testdf,
                            directory=test_dir,
                            x_col="Image",
                            y_col=None,
                            batch_size=1,
                            seed=42,
                            shuffle=False,
                            class_mode=None,
                            target_size=(image_size, image_size))

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//8
STEP_SIZE_VALID=valid_generator.n//8
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Input
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
import zipfile
import seaborn as sns
from os import listdir, makedirs
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
model = Sequential([
               tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(image_size, image_size, 3)),
                    layers.Conv2D(16, 3, padding='same', activation='relu'), # convolution 2D
                    layers.MaxPooling2D(), # extracting important features
                    layers.Conv2D(32, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Conv2D(64, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Flatten(),
                    layers.Dense(4300, activation='relu'),
                    layers.Dense(4251)]) # layers of one hot encoding 
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

filepath="weights.h5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
callbacks_list = [checkpoint, es]

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50, verbose=1,
                  shuffle = True, callbacks=callbacks_list)
gc.collect()

In [ ]:
# Xception

inputs = Input((299, 299, 3))
backbone = Xception(input_tensor=inputs, include_top=False, weights="imagenet")
backbone.trainable = False
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(4300, activation="relu")(x)
x = Dense(4251, activation="softmax")(x)

model = Model(inputs, x)
lr = 1e-4
optimizer = Adam(lr)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//8
STEP_SIZE_VALID=valid_generator.n//8
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

In [ ]:
filepath="weights.h5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                  shuffle = True, callbacks=callbacks_list)

In [ ]:
scores = model.evaluate_generator(generator=valid_generator)
print("Accuracy = ", scores[1])

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(image_size,image_size, 3))
base_model.trainable = False ## Not trainable weights

base_model.summary()

In [ ]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(4251, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)


filepath="weights.h5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
callbacks_list = [checkpoint, es]

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2,
                  shuffle = True, callbacks=callbacks_list)